In [98]:
#system
import sys
import os

#data processing
import numpy as np
import pandas as pd
from numpy import genfromtxt

#flow and solute transport
import flopy
from copy import deepcopy as deepcopy

#geostatistics
import gstools as gs
import skgstat as skg
from skgstat import OrdinaryKriging
from pykrige.ok import OrdinaryKriging

import scipy
from scipy.stats import gmean

#parameter--space sampling
from skopt.sampler import Lhs
import time

#plotting
from skimage.transform import resize
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

#additional plotting functionallity
import matplotlib.ticker as mtick
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Arial']})
plt.rcParams['font.size'] = 16
from matplotlib import cm
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.offsetbox import AnchoredText
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.widgets import RadioButtons, Slider

#version checks
print(skg.__version__)
print(gs.__version__)
print(flopy.__version__)


ModuleNotFoundError: No module named 'skimage'

### Functions

In [13]:
#3d plotting function for 2d visuals
def plot_2d(map_data, dx, dy, colorbar_label, title, cmap, clim, invert):
    # shape information
    r, c = np.shape(map_data)
    # define plot coordinates
    x_coord = np.linspace(0, dx*c, c+1)
    y_coord = np.linspace(0, dy*r, r+1)
    
    X, Y = np.meshgrid(x_coord, y_coord)

    plt.figure(figsize=(8, 4), dpi=200)
    plt.pcolormesh(X, Y, map_data, cmap=cmap, shading = 'auto', edgecolor ='k', linewidth = 0.05)
    plt.gca().set_aspect('auto')  #changed from equal
    # add a colorbar
    cbar = plt.colorbar() 
    # plt.clim(cmin, cmax) 
    # label the colorbar
    cbar.set_label(colorbar_label, fontsize = 14)
    cbar.ax.tick_params(labelsize=12) 
    plt.tick_params(axis='both', which='major', labelsize = 12)
    plt.xlim((0, dx*c)) 
    plt.ylim((0, dy*r)) 
    plt.title(title, fontsize = 14)
    plt.xlabel('x distance [m]', fontsize = 14)
    plt.ylabel('y distance [m]', fontsize = 14)
    if invert == 1:
        plt.gca().invert_yaxis()
    plt.clim(clim)
    plt.show()

In [ ]:
#Slice Viewer

def remove_keymap_conflicts(new_keys_set):
    for prop in plt.rcParams:
        if prop.startswith('keymap.'):
            keys = plt.rcParams[prop]
            remove_list = set(keys) & new_keys_set
            for key in remove_list:
                keys.remove(key)
                
def multi_slice_viewer(volume):
    remove_keymap_conflicts({'j', 'k'})
    fig, ax = plt.subplots()
    ax.volume = volume
    ax.index = volume.shape[0] // 2
    ax.imshow(volume[ax.index])
    fig.canvas.mpl_connect('key_press_event', process_key)

def process_key(event):
    fig = event.canvas.figure
    ax = fig.axes[0]
    if event.key == 'j':
        previous_slice(ax)
    elif event.key == 'k':
        next_slice(ax)
    fig.canvas.draw()

def previous_slice(ax):
    volume = ax.volume
    ax.index = (ax.index - 1) % volume.shape[0]  # wrap around using %
    ax.images[0].set_array(volume[ax.index])

def next_slice(ax):
    volume = ax.volume
    ax.index = (ax.index + 1) % volume.shape[0]
    ax.images[0].set_array(volume[ax.index])

In [ ]:
#voxel cross-sections

#https://terbium.io/2017/12/matplotlib-3d/
IMG_DIM = 50
resized = resize(transformed, (IMG_DIM, IMG_DIM, IMG_DIM), mode='constant')


def explode(data):
    shape_arr = np.array(data.shape)
    size = shape_arr[:3]*2 - 1
    exploded = np.zeros(np.concatenate([size, shape_arr[3:]]), dtype=data.dtype)
    exploded[::2, ::2, ::2] = data
    return exploded

def expand_coordinates(indices):
    x, y, z = indices
    x[1::2, :, :] += 1
    y[:, 1::2, :] += 1
    z[:, :, 1::2] += 1
    return x, y, z

def plot_cube(cube, angle=320):
    
    facecolors = cm.viridis(cube)
    facecolors[:,:,:,-1] = cube
    facecolors = explode(facecolors)
    
    filled = facecolors[:,:,:,-1] != 0
    x, y, z = expand_coordinates(np.indices(np.array(filled.shape) + 1))

    fig = plt.figure(figsize=(30/2.54, 30/2.54))
    ax = fig.gca(projection='3d')
    ax.view_init(30, angle)
#     ax.set_xlim(right=IMG_DIM*2)
#     ax.set_ylim(top=IMG_DIM*2)
#     ax.set_zlim(top=IMG_DIM*2)
    
    ax.voxels(x, y, z, filled, facecolors=facecolors, shade=False)
    plt.show()


# Statistics: data and pre-processing

In [9]:
#Model Grid with head values for wells pumping at 500gpm

heads500 = pd.read_csv('C:\\Users\\willg\\OneDrive\\Documents\\2_School\\GraduateSchool\\Fall2021\\!Research\\Geostatistics_paper\\ModelData\\Airport_heads500gpm.csv')
#observing dataframe information if need be

#converting pandas dataframe to numpy array
heads500_array = heads500.to_numpy()
# ksat_data.shape

x = np.array(heads500_array[1:,0])
y = np.array(heads500_array[1:,1])
head500_data = np.array(heads500_array[1:,2])

In [ ]:
#head visualization

In [10]:
#500 randomly sampled points across model boundary (Each point contains full depth profile)
airprt_params = pd.read_csv('C:\\Users\\willg\\OneDrive\\Documents\\2_School\\GraduateSchool\\Fall2021\\!Research\\Geostatistics_paper\\ModelData\\Airport_modelParams.csv' )
airprt_params.describe()

# airprt_params_array = heads500.to_numpy()

,x,y,z,ksat,org_mat
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,47.818000,37.506000,4.500000,31.412495,10.571168
std,27.712785,22.362244,2.872569,31.302641,22.532083
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,17.000000,2.000000,9.000000,0.250000
50%,47.500000,37.500000,4.500000,22.000000,0.750000
75%,72.000000,59.000000,7.000000,33.941000,1.500000
max,96.000000,73.000000,9.000000,141.000000,65.000000


In [ ]:
#measured data
loc = np.arange(['B004', 'B004', 'B004', 'B005', 'B005', 'B006', 'B006', 'B006'])
depth = np.arange(['1', '7', '13', '8', '14', '3', '9', '13']) #feet
hyd_cond = np.arange(['10.454', '12.442', '1.058', '14.602', '14.602', '2.906', '24.970', '18.1656']) #m/day

### Parameter Space

In [15]:
lhs = Lhs(lhs_type="classic", criterion=None)

In [131]:
#Parameters Ps[0-...]
#Ps[0] = log variance
#Ps[1,2,3] = correlation length x,y,z
#Ps[4,5,6] = rotation angles + an angle tolerance of BLANK degrees -- specify in angle_tol
#Ps[7] = anisotropy
Ps = lhs.generate([(-2., -0.5),
                   (1.0, 50.), (1.0, 100.0), (1.0, 50.0),
                   (0.0, 1.57), (0.0, 1.57), (0.0, 1.57), 
                   (0.0, 0.5)], 15)


In [127]:
#model dimensions

# x = y = np.arange(100)
x = np.arange(50)
y = np.arange(50)
z = np.arange(50)

### Realizations

In [83]:
%matplotlib qt
start_td = time.time() # start a timer

for i in range(0, 2):
    p = Ps[i]
    
    model = gs.Exponential(dim=3, 
                           var=10**p[0], 
                           len_scale=[p[1], p[2], p[3]], 
                           nugget = 0,
                           anis = p[7],
                           angles=[p[4], p[5], p[6]],
                           integral_scale = None,
                           rescale=None,
                           latlon=False,
                           var_raw=None,
                           hankel_kw=None,
#                            **opt_arg,
                          )
    
    srf = gs.SRF(model)#, seed=20170519)
    
    field = srf.structured([x, y, z])
    
    #kriging measured ksat data points
    
    
    end_td = time.time() # end timer
    print('Sec to run generate field: ', (end_td - start_td)) # show run time
#     print(np.log10(np.max(field)/np.min(field)))

    #plotting
    fig, ax = plt.subplots(1, 2, figsize=[10, 5], gridspec_kw={'width_ratios': [2, 1]})
    
#     ax[0].figure(figsize=(5, 4), dpi=200)
    ax[0].pcolormesh(field[:, :, 0])
    ax[0].set_aspect('equal')
    ax[0].set_xlabel('x')
    ax[0].set_ylabel('y')
    
#     cbar = ax[0].colorbar()
#     cbar.set_label('mD')
    
#     ax[1].figure(figsize=(5, 4), dpi=200)
    im = ax[1].pcolormesh(field[0, :, :])
    ax[1].set_aspect('equal')
    ax[1].set_xlabel('z')
    
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.set_label('variance')
    
#     divider = make_axes_locatable(ax[1])
#     cax = divider.append_axes("bottom", size = '5%', pad=0.05)
#     cbar = fig.colorbar(im, ax=ax[1], cax=cax)
#     cbar.set_label('variance')


Sec to run generate field:  2.271435022354126
Sec to run generate field:  4.550974130630493


# Plotting

In [132]:
#new plotting attempt on one field --- generate one field
%matplotlib qt

p = Ps[14]   
model = gs.Exponential(dim=3, 
                        var=10**p[0], 
                        len_scale=[p[1], p[2], p[3]], 
                        nugget = 0,
                        anis = p[7],
                        angles=[p[4], p[5], p[6]],
                        integral_scale = None,
                        rescale=None,
                        latlon=False,
                        var_raw=None,
                        hankel_kw=None)
    
srf = gs.SRF(model)#, seed=20170519)
    
field = srf.structured([x, y, z])


In [133]:
fig1 = multi_slice_viewer(field.T) #xy
fig2 = multi_slice_viewer(field) #y+depth